# Convolution, Filtrage.

## Définition

La convolution est une opération mathématique qui correspond à ce que les physiciens appellent un filtrage. 
On peut montrer qu'un très grand nombre d'opérations de traitement du son et des images sont des convolutions, on peut citer par exemple : le réhaussement ou l'aténuation de certaines fréquences (equalizer), la création d'un écho (réverbération),réalaliser une moyenne glissante sur un signal, ou une moyenne locale sur une image, le réhaussement de contraste d'une image, la création d'un flou etc...
https://fr.wikipedia.org/wiki/Noyau_(traitement_d%27image)
Les réseaux de neurones les plus performants actuellement dans de nombreux domaines du traitements d'images sont des CNN (Convolutional Neural Network) et n'effectue pour ainsi dire que des convolutions entre chaque couche du réseau :
https://fr.wikipedia.org/wiki/Réseau_neuronal_convolutif

Ce sont également des formules de convolution qui permettent de reconstruire un signal analogique à partir d'un signal numérique (Nous verrons le théorème de Shannon plus tard.)

Il est possible de définir la convolutions de fonctions définies sur $\mathbb{R}$ :
\begin{equation}
h(t):=f\star g(t)=\int_{u\in\mathbb{R}}f(u)g(t-u)du=\int_{u\in\mathbb{R}}f(t-u)g(u)du
\end{equation}
La fonction $h$ est dite la convoluée de $f$ et de $g$. Notez que la définition est symétrique en $f$ et $g$. on peut le voir en effectuant le changement de variable $y=t-u$.

On peut également définir la convolution de deux fonctions $2\pi$-périodiques $f$ et $g$:
\begin{equation}
h(t):=f\star g(t)=\frac{1}{2\pi}\int_{u=0}^{2\pi}f(u)g(t-u)du=\frac{1}{2\pi}\int_{u=0}^{2\pi}f(t-u)g(u)du
\end{equation}
Et on peut définir la notion de convolution sur des vecteurs complexes ayant $N$ composantes ($\mathbb{C}^N$).
Si $f$ et $g$ sont deux vecteurs de $\mathbb{C}^N$, on dit que $h$ est le vecteur convolué ou le produit de convolution de $f$ et $g$ quand il est défini de la manière suivante :
\begin{equation}
\forall k\leqslant N-1,\,h[k]=\sum_{n=0}^{N-1}f[n]g[k-n]
\end{equation}
Cette définition pose un problème quand on veut évaluer $g[k-n]$ quand $n>k$... Il existe pour faire simple deux conventions classiques pour résoudre ce problème :
\begin{enumerate}
\item Pour $l<0$, on pose $g[l]=0$.
\item On considère que $f$ et $g$ sont périodiques de période $N$ et si $l<N$ on dit que $g[l]=g[N+l]$.
\end{enumerate}
On parle alors de convolution circulaire. C'est cette seconde définition de la convolution que nous utiliserons dans le cours et dans les notebooks. Nous verrons que la convolution circulaire est intimement liée à la transformée de Fourier. Ceci est vrai pour les signaux numériques mais également pour les fonctions, $2\pi$-périodiques ou pas. 

La formule qui est donnée est la convolution 1D, celle utilisée pour les sons ou les signaux 1D. On peut étendre cette définition aux images et aux tableaux 2D : si $f$ et $g$ sont deux tableaux de nombres on note $h=f\star g$ le produit de convolution de ces deux tableaux 2D par 
\begin{equation}
h[k_1][k_2]=\sum_{n_1}\sum_{n_2}f[n_1,n_2]g[k_1-n_1,k_2-n_2].
\end{equation}


## Lien entre la convolution circulaire et la transformée de Fourier.

On rappelle la définition de la transformée de Fourier discrète. 
Cette transformée est une application qui envoie un vecteur de $\mathbb{C}^N$ sur un vecteur de $\mathbb{C}^N$, où $\mathbb{C}^N$ est muni du produit scalaire suivant 
$$\langle f,g\rangle=\sum_{k=0}^{N-1}f[k]\overline{g[k]}.$$
Cette application linéaire est définie par la formule suivante : pour tout $k\in\{ 0: N-1\}$
$$\hat{f}[k]=\sum_{n=0}^{N-1}f[n]e^{-\frac{2i\pi kn}{N}}$$
Je vous laisse vous reporter au notebook précédent pour plus d'explications et pour le calcul de cette transformée en python.

Si $h$ est le produit de convolution de $f$ et $g$ alors pour tout $k\in\{ 0: N-1\}$
\begin{equation}
\hat h[k]=\hat f[k]\hat g[k]
\end{equation}
Ainsi le produit de convolution circulaire qui a une formule un peu compliquée, s'exprime de manière très simple dans le domaine de Fourier.  
Démonstration :
\begin{align*}
\hat h[k]&=\sum_{n}h[n]e^{-\frac{2i\pi kn}{N}}\\
&=\sum_{n}\sum_{l}f[l]g[n-l]e^{-\frac{2i\pi kn}{N}}\\
&=\sum_{l}f[l]\sum_n g[n-l]e^{-\frac{2i\pi kn}{N}}\\
&=\sum_{l}f[l]\sum_p g[p]e^{-\frac{2i\pi k(p+l)}{N}}\\
&=\sum_{l}f[l]e^{-\frac{2i\pi kl}{N}}\sum_p g[p]e^{-\frac{2i\pi kp}{N}}\\
&=\hat f[k]\times \hat g[k]. 
\end{align*}

Ainsi, il est possible d'effectuer la convolution de deux signaux en effectuant le produit dans le domaine de Fourier.   

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import numpy as np
import param
import holoviews as hv,panel as pn,param
from holoviews.streams import Pipe
import time
import pandas as pd
import panel as pn
from panel.pane import LaTeX
from scipy.io.wavfile import read
from scipy import fftpack
from IPython.display import Audio
import requests
from io import BytesIO
from PIL import Image
import shutil
from urllib.request import urlopen
import io
from scipy.io.wavfile import read
import scipy.io as sio

In [ ]:
S4=res=np.load('Blocks.npy')
S3=res=np.load('Piece.npy')
signauxRef= {"Piece" : S3,"Blocks" : S4}



Nous allons voir sur quelques exemples simples l'effet de la convolution.

In [ ]:
def Gaussienne(N,sigma2,b):
    x = np.linspace(-b, b, N)
    y=np.exp(-np.abs(x)*np.abs(x)/sigma2)
    return y
def Porte(N,d,f):
    y=np.zeros(N)
    for k in range(d,f):
        y[k]=1
    return y
def Peigne(N,d):
    y=np.zeros(N)
    nb=int(np.floor(N/d))
    for k in range(0,nb):
        y[k*d]=1
    return y    

In [ ]:
g=Gaussienne(1024,0.1,10)
po=Porte(1000,300,600)
pe=Peigne(1000,50)


Pour calculer la convolution entre deux signaux (ou vecteurs) une manière simple est de faire le produit dans le domaine de Fourier. Théoriquement la convolution de deux signaux réels est réel, mais si on passe par Fourier, on peut avoir des erreurs numériques complexes donc je vous invite à ne conserver que la partie réelle du résultat. Voici un exemple de la convolution d'un signal de référence avec une gaussienne :


In [ ]:
FS=fftpack.fft(S3)
Fg=fftpack.fft(g)
conv=np.real(fftpack.ifft(FS*Fg))
pn.Column(hv.Curve(S3).opts(width=800,title='signal de référence'),\
       hv.Curve(g).opts(width=800,title='gaussienne'),hv.Curve(conv).opts(width=800,title='produit de convolution'))

In [ ]:
def Convcirc(S1,S2):
    y=np.real(fftpack.ifft(fftpack.fft(S1)*fftpack.fft(S2)))
    return y

Même si la convolution est symétrique, on a coutume, quand un des deux signaux est élémentaire de parler de fitre pour ce dernier. Ici on a ainsi convoluer le signal de référence par un filtre gaussien. On retrouve la forme générale du signal de référence mais en plus lisse et en décalé. Ce lissage est dû à la forme de la gaussienne et le décalage est dû au fait que la gaussienne utilisée est centrée au milieu, ici en 512. Attention ici j'ai pris soin de prendre deux siganux de même taille. Si tel n'est pas le cas, il faut compléter le plus petit par des zéros.
Pour voir l'effet des paramètres de la gaussienne, faites des tests avec le dashboard suivant :

In [ ]:
class ConvGaussienne(param.Parameterized):
    sig = param.ObjectSelector(default="Piece",objects=signauxRef.keys())
    pos = param.Integer(100,bounds=(0,1023))
    var = param.Number(0.3,bounds=(0.1,2))
    def view(self):
        options = dict(width=500,height=150,toolbar=None,xaxis=None,yaxis=None)
        s=signauxRef[self.sig]
        fs=fftpack.fft(s)
        g=Gaussienne(1024,self.var**2/10,10)
        somme=np.sum(g)
        g2=np.roll(g/somme,self.pos-512)
        fg2=fftpack.fft(g2)
        conv=np.real(fftpack.ifft(fs*fg2))
        return pn.Column(hv.Curve(s).opts(**options).opts(title='signal de référence'),hv.Curve(g2).opts(color='red').opts(title='filtre gaussien').opts(**options)\
                         ,hv.Curve(conv).opts(color='black').opts(**options).opts(title='produit de convolution'))

In [ ]:
cg= ConvGaussienne()
pn.Row(cg.param,cg.view)

## La convolution comme une moyenne glissante.

Vous pouvez voir la convolution comme une moyenne glissante ou moyenne locale... quand on effectue la convolution par une gaussienne on effectue en fait une moyenne locale. 
Pour mieux comprendre cet effet de moyenne local on peut regarder plus précisément l'effet de la convolution par une fonction indicatrice ou fonction porte.



In [ ]:
class ConvInd(param.Parameterized):
    sig = param.ObjectSelector(default="Piece",objects=signauxRef.keys())
    pos = param.Integer(0,bounds=(0,35))
    Nbmoy = param.Integer(5,bounds=(1,30))
    def view(self):
        options = dict(width=600,height=150,toolbar=None,xaxis=None,yaxis=None)
        s=signauxRef[self.sig]
        fs=fftpack.fft(s)
        po=Porte(1024,self.pos,self.pos+self.Nbmoy)
        somme=np.sum(po)
        fpo=fftpack.fft(po/somme)
        conv=np.real(fftpack.ifft(fs*fpo))
        return pn.Column(hv.Curve(s).opts(**options).opts(title='signal de référence')\
                         ,hv.Bars(po[0:65]).opts(color='red',title='filtre porte').opts(**options)\
                         ,hv.Curve(conv).opts(color='black').opts(**options).opts(title='produit de convolution'))

Dans l'exemple suivant, on filtre un signal de référence par une focntion indicatrice (ou focntion porte). le second graphique ne représente que les 55 premières composantes de ce filtre. Les valeurs par défaut pos=0 et Nbmoy correspondent à un signal nuémrique (vecteur) ayant ses 5 premières composantes non nulles et égales à 1/5...
Faire la convolution par un tel sognal revient à calculer une moyenne glissante sur 5 points à partir du signal de départ. Si on augmente NbMoy, on augmente le nombre de points surle quel on fait la moyenne et on la convolution lisse encore plus le signal original. Si on décale (avec le paramètre pos) la focntion porte, on effectue une moyenne glissante décalée... on translate docn juste le produit de convolution.

J'ai utilisé des bars pour l"indicatrice juste pour aider à la visualisation mais ce choix n'a aucun impact sur le résultat.

In [ ]:
cp= ConvInd()
pn.Row(cp.param,cp.view)

Nous pouvons bien entendu effectuer des moyennes glissantes avec des coefficients non égaux : par exemple on peut faire une moyenne sur trois points en prenant comme coefficients $\frac{1}{2},\frac{1}{3}$ et $\frac{1}{6}$.
Si on considère le vecteur (ou signal numérique) suivant 
\begin{equation}
S=[0,0,6,24,0,18,-6,30,0,0,12]
\end{equation}
et on calcule la moyenne glissante.
Pour caluler chaque terme de la moyenne glissante $M[k]$ on utilise la règle suivante :
\begin{equation}
M[k]=\frac{1}{2}S[k]+\frac{1}{3}S[k-1]+\frac{1}{6}S[k-2]
\end{equation}
Pour $k=0$ et 1 vous allez avoir un problème... que vaut $S[-1]$ ou $S[-2]$ ? Comme je l'ai expliqué plus haut on va faire l'hypothèse que le signal est périodique et utiliser les deux dernières valeurs du signal, ainsi $S[-1]=12$ et $S[-2]=0$. 
La moyenne glissante (circulaire) sera 
\begin{equation}
M=[4,2,3,14,9,13,3,16,9,5,6]
\end{equation}
Vous pouvez vérifier que le résulat est celui qui est donné par le calcul suivant :

In [ ]:
S=[0,0,6,24,0,18,-6,30,0,0,12]
filtre=[1/2,1/3,1/6,0,0,0,0,0,0,0,0]
FS=fftpack.fft(S)
Ff=fftpack.fft(filtre)
conv=np.real(fftpack.ifft(FS*Ff))
print(conv)

Dans cet exemple on a effectué à la main la convolution par le filtre qu'on peut représenter graphiquement par :

In [ ]:
hv.Bars(filtre)

Dans l'exemple sur les focntions "porte" ou "indicatrice", on a effectué une moyenne glissante par un filtre de la forme  

In [ ]:
po=Porte(11,0,5)
hv.Bars(po).opts(color='red')

Dans le cas d'une convolution par une gaussienne, on a effectué une moyenne glissante avec des coefficients ressemblant à 

In [ ]:
options = dict(width=800,height=150,toolbar=None,xaxis=None)
g=Gaussienne(1024,0.1,10)
hv.Bars(g[470:550]).opts(**options)

L'élément neutre de la convolution est le vecteur n'ayant que des composantes nulles sauf la première qui est égale à 1. Ce vecteur est parfois appelé Dirac ou Dirac discret en hommage à Paul Dirac grand mathématicien du 20ème siècle. 
Essayez de le vérifier sur quelques exemples.

## Convolution et echos

La création d'un ou plusieurs échos sur un signal numérique peut se faire par convolution par un signal, somme de Dirac décalés...

In [ ]:
Diracs=np.zeros(1024)
Diracs[0]=1
Diracs[100]=1/2
Diracs[600]=-1/2
g=Gaussienne(1024,0.1,10)
g2=g/sum(g)*5
Prodconv=Convcirc(g2,Diracs)
pn.Column(hv.Bars(Diracs).opts(**options).opts(title='Diracs'),\
          hv.Curve(g2).opts(color='red',title='Gaussienne').opts(**options),
          hv.Curve(Prodconv).opts(**options,title='Produit de convolution'))

On peut interpréter ce produit de convolution de deux manières différentes, toutes les deux valables :
\begin{enumerate}
\item On peut voir les Dirac comme le signal et la gaussienne comme le filtre, dans ce cas, on peut voir le produit de convolution comme une version lissée et décalé du signal "Diracs" (décalé car la gaussienne est centréece qui induit une translation circulaire de la moitié de la longueur du signal).
\item On peut voir la Gaussienne comme le signal et les Dirac comme le filtre. Dans ce cas on peut voir le produit de convolution comme des copies décalées de la Gaussienne... le décalage et l'amplitude des copies étant définies par l'amplitude et la dispositions des coefficients non nuls dans le signal "Diracs". Vous pouvez essayer de faire des tests pour vérifier.   
\end{enumerate}

Question : Effectuer des tests similaires en remplaçant la gaussienne par une fonction porte et en modifiant le vecteur Diracs. Essayer d'anticiper le résultat pour voir si ce dernier correspond à ce que vous avez compris.

## Convolution et périodisation d'un signal.

Pour les raisons précédentes, la convolution avec un peigne de Diracs, c'est-à-dire des Diracs régulièrement espacés (le premier paramètre de la fonction indique le nombre de composantes du vecteurs et le second l'espacement entre les "dents" du peigne) : 

In [ ]:
peigne=Peigne(1024,128)
hv.Bars(peigne).opts(**options)

Modifier le second paramètre pour voir... 
Dans la suite, on prendra toujours un espacement qui est un diviseur du nombre de composantes du signal.

La convolution par un tel peigne de Dirac permet de périodiser un signal :


In [ ]:
gper=Convcirc(g2,peigne)
pn.Column(hv.Bars(peigne).opts(**options).opts(title='peigne de Dirac'),\
          hv.Curve(g2).opts(color='red',title='Gaussienne').opts(**options),
          hv.Curve(gper).opts(**options,title='Gaussienne périodisée'))

Questions : Que se passe t il si on rapproche les dents du Peigne ? si on les espace ?
Essayez de remplacer la gaussienne par un autre signal ou filtre. 

# Un aperçu de la convolution pour le traitement d'images.

In [ ]:
local=0
def chargeData(name):
    if local:
        if name=='Lenna':
            res=np.array(Image.open("./Archive/img/Lenna.jpg")).astype(float)
        if name=='Canaletto':
            res=np.array(Image.open("./Archive/img/Canaletto.jpeg")).astype(float)
        if name=='Minotaure':
            res=np.array(Image.open("./Archive/img/MinotaureBruite.jpeg")).astype(float)   
        if name=='Cartoon':
            res=np.array(Image.open("./Archive/img/Cartoon.jpg")).astype(float) 
    else:
        if name=='Lenna':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Lenna.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Canaletto':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Canaletto.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Minotaure':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/MinotaureBruite.jpeg'
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
        if name=='Cartoon':
            url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Cartoon.jpg'        
            response = requests.get(url)
            res=np.array(Image.open(BytesIO(response.content))).astype(float)
    return res

In [ ]:
im=chargeData('Canaletto')
im2=chargeData('Lenna')
imagesRef= {"Lenna" : im2,"Canaletto" : im}
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)
pn.Row(hv.Image(im).opts(**options),hv.Image(im2).opts(**options))

In [ ]:
def Gaussian2D(N,sigma):
    x, y = np.meshgrid(np.linspace(-1,1,N), np.linspace(-1,1,N))
    d = np.sqrt(x*x+y*y)
    mu =0.0
    g= np.exp(-( (d-mu)**2 / ( 2.0 * sigma**2 ) ) )
    g2=np.fft.fftshift(g)
    s=sum(sum(g2))
    g2=g2/s
    return g2

Pour des images 2D, la convolution s'effectue via la convolution 2D.

In [ ]:
def Convcirc2D(I1,I2):
    y=np.real(fftpack.ifft2(fftpack.fft2(I1)*fftpack.fft2(I2)))
    return y

Plus le noyau gaussien est large plus le flou est large.

In [ ]:
optionsImage = dict(cmap='gray',xaxis=None,yaxis=None,width=450,height=450,toolbar=None)
h_1=Gaussian2D(512,0.005)
h_2=Gaussian2D(512,0.01)
h_3=Gaussian2D(512,0.02)
imconv1=Convcirc2D(im,h_1)
imconv3=Convcirc2D(im,h_3)
pn.Row(hv.Image(imconv1).opts(**optionsImage),hv.Image(imconv3).opts(**optionsImage))

Si on veut détecter les contours d'une image, une possibilité est de lui appliquer un filtre spécifique c'est à dire effectuer une convolution particulière : La variable vois (pour voisinage) défini le nombre de pixels sur lequel on cherche à détecter les contours. Vous pouvez modifier sa valeur à 2 ou 3...

In [ ]:
Filtrecontours=0*im
vois=1
for k in np.arange(0,1+2*vois):
    for j in np.arange(0,1+2*vois):
        Filtrecontours[k,j]=-1
Filtrecontours[vois,vois]=(2*vois+1)**2-1
Contours=Convcirc2D(im,Filtrecontours)
hv.Image(np.abs(Contours[1+vois:511-vois,1+vois:511-vois])).opts(**optionsImage)

In [ ]:
Filtrecontours=0*im2
vois=1
for k in np.arange(0,1+2*vois):
    for j in np.arange(0,1+2*vois):
        Filtrecontours[k,j]=-1
Filtrecontours[vois,vois]=(2*vois+1)**2-1
Contours=Convcirc2D(im2,Filtrecontours)
hv.Image(np.abs(Contours[1+vois:511-vois,1+vois:511-vois])).opts(**optionsImage)

La convolution permet ainsi de mettre en lumière certaines inforamtions importantes dans une image. C'est pour cette raison qu'elle est abondamment utilisée dans les réseaux de neurones convolutif, CNN en anglais.